In [1]:
import os
import time
import re
import glob
import pandas as pd
import requests
        
def get_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    names = []
    for b in blobs: names.append(b.name)
    return names

def upload_to_output(path, bucket_name, folder_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(folder_name + '/' + path.split('/')[-1])
    blob.upload_from_filename(path)
    
def stack_tweets(new_records, existing_records):
    captured_tokens = list(existing_records.id.unique())
    response_tokens = list(new_records.id.unique())
    already_captured = [x for x in response_tokens if x in captured_tokens]
    
    x = len(captured_tokens)
    y = len(response_tokens)
    z = len(already_captured)
    
    existing_records = existing_records.append(new_records, ignore_index=False)
    
    print('Existing tokens: ', x, 'Response tokens: ', y, 'Already_captured: ', z, 'Efficiency Rate: ', str(round(z/y,2)*100)+'%')
    return existing_records

In [2]:
from google.cloud import storage

bucket_name = 'sw-airlines-data-hub'
folder_name = 'data/source/'
os.system('mkdir data/')
os.system("gsutil -m cp -r 'gs://sw-airlines-data-hub/data/source/*HOUR*' data/") 

mkdir: cannot create directory ‘data/’: File exists
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-10_#southwestairlines.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-10_#southwestairline.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-10_@SouthwestAir.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-11_#airlines.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-10_#airlines.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-11_#southwestairline.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-11_#southwestairlines.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-11_@SouthwestAir.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-14_#airlines.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-14_#southwestairline.csv...
Copying gs://sw-airlines-data-hub/data/source/2022-10-1_HOUR-14_#southwestairlines.csv..

0

In [3]:
first = True
result = pd.DataFrame()
import glob

for y in glob.glob('data/*'):
    try:
        if first:
            result = pd.read_csv(y)
            result = result.set_index('id')
            first = False
        else:
            new_result = pd.read_csv(y)
            new_result = new_result.set_index('id')
            result = result.append(new_result, ignore_index=False)
        print(len(result))
    except Exception:
        print('Invalid file: ' , y)

raw_records = len(result)
!rm data/*
# print('Raw Tweet Records Length: ', len(result))

300
492
680
780
1081
1281
1381
1582
1782
1983
2183
2484
2684
2876
3177
3470
3570
3670
3770
3970
4067
4368
4468
4568
4667
4868
5068
5268
5465
5665
5765
5962
6263
6363
6545
6846
7145
7245
7446
7646
7847
8039
8340
8529
8730
8830
9029
9230
9530
9630
9831
10112
10298
10498
10598
10784
10973
11273
11373
11668
11868
12169
12369
12569
12763
13063
13163
13263
13456
13556
13756
13956
14149
14249
14349
14539
14639
14827
14927
15127
15226
15527
15727
15927
16027
16328
16529
16729
16915
17106
17300
17501
17600
17801
17901
18001
18200
18500
18700
18800
19100
19300
19399
19700
19901
20101
20301
20501
20601
20801
21000
21100
21300
21399
21588
21889
22087
22379
22564
22664
22864
23164
23364
23464
23665
23966
24066
24266
24466
24566
24766
25066
25265
25365
25565
25866
26167
26267
26566
26866
26966
27165
27358
27659
27952
28052
28152
28452
28752
29051
29351
29451
29551
29651
29846
30043
30243
30343
30542
30642
30842
31043
31224
31424
31625
31724
31824
32023
32223
32423
32523
32722
32822
33021
33210
33310

In [19]:
result['tweet_token'] = result.index

In [20]:
result

,text,author_id,in_reply_to_user_id,conversation_id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,edit_history_tweet_ids,withheld.copyright,withheld.country_codes,tweet_token
id,,,,,,,,,,,,,
1571409179830587393,@ywprep @SouthwestAir @IfThenSheCan Am inspire...,1470578806058590209,3.870538e+09,1400856167132368902,2022-09-18T08:01:58.000Z,0,0,0,0,NaN,NaN,NaN,1571409179830587393
1571407508778156032,@FireGregRoman_ @Ravens @SouthwestAir @Patrick...,1332604242675130368,1.330829e+18,1571182195443646464,2022-09-18T07:55:20.000Z,0,0,0,0,NaN,NaN,NaN,1571407508778156032
1571402498178691073,When I picked up my bag in baggage claim it lo...,47133143,NaN,1571402498178691073,2022-09-18T07:35:25.000Z,0,0,0,0,NaN,NaN,NaN,1571402498178691073
1571392419458912257,"@ILickety @SouthwestAir Yeah, that first one w...",1433359597117530115,1.305605e+18,1569687344067842049,2022-09-18T06:55:22.000Z,0,0,0,0,NaN,NaN,NaN,1571392419458912257
1571388999641464833,@JudgeSmailsGolf @tkcaddie @SouthwestAir @lagu...,2991985492,7.361791e+08,1571361678335303680,2022-09-18T06:41:47.000Z,0,0,0,0,NaN,NaN,NaN,1571388999641464833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569967578784870400,#914の日\nサウスウエスト航空 N914WN B737-7H4(WL)\n#KPHX #...,241550532,NaN,1569967578784870400,2022-09-14T08:33:34.000Z,0,0,2,0,NaN,NaN,NaN,1569967578784870400
1569906904146784257,@SouthwestAir goes everywhere \n#adventureswit...,3193625618,7.212562e+06,1569906904146784257,2022-09-14T04:32:28.000Z,1,0,1,0,NaN,NaN,NaN,1569906904146784257
1569906520883884032,$LUV in Uptrend: price may ascend as a result ...,374729426,NaN,1569906520883884032,2022-09-14T04:30:56.000Z,0,0,0,0,NaN,NaN,NaN,1569906520883884032


In [21]:
export = result.copy()
unique_number = export.index.nunique()
print('Number of Tweets After DeDupe:', unique_number, 'Reduction:', str(round(1-unique_number/raw_records,3)*100)+'%')

Number of Tweets After DeDupe: 47785 Reduction: 90.3%


In [22]:
export = export.drop_duplicates()
len(export)

55287

In [23]:
export.to_csv('sw-airlines-tweets-base.csv', index=True)

In [24]:
result.to_csv('sw-airlines-tweets-base-over-time.csv', index=True)

In [25]:
upload_to_output('sw-airlines-tweets-base.csv', bucket_name, 'data/processed')
upload_to_output('sw-airlines-tweets-base-over-time.csv', bucket_name, 'data/processed')

In [26]:
user_df = pd.read_csv('gs://sw-airlines-data-hub/data/users/user_id.csv')
user_df = user_df.rename(columns={'id': 'author_id', 
                                  'public_metrics.followers_count':'user.followers_count',
                                  'public_metrics.following_count':'user.following_count',
                                  'public_metrics.tweet_count':'user.tweet_count',
                                  'public_metrics.listed_count':'user.listed_count'})
user_df.head()

,author_id,name,verified,description,created_at,profile_image_url,username,user.followers_count,user.following_count,user.tweet_count,user.listed_count,withheld.country_codes
0,284837895,Roy Winkelman,False,NaN,2011-04-20T02:02:52.000Z,https://pbs.twimg.com/profile_images/151649800...,roy_winkelman,74,246,5933,2,NaN
1,138182790,Maga,False,que felicidad poder tener una vara magica y sa...,2010-04-28T21:23:31.000Z,https://pbs.twimg.com/profile_images/121151766...,magallyt,125,187,7663,2,NaN
2,84443638,jrreign13,False,hockey. that's it. GKG,2009-10-22T22:43:19.000Z,https://pbs.twimg.com/profile_images/644311136...,JRReign13,55,473,1231,2,NaN
3,2521101594,Carmen Ye,False,💼People & Culture @theactionnet 🎓@umich @cal &...,2014-05-24T18:49:51.000Z,https://pbs.twimg.com/profile_images/137345132...,carmenye_,522,1156,8327,12,NaN
4,15947342,Shaun Lysen,False,"Data scientist. Sports, history, science, econ...",2008-08-22T17:06:52.000Z,https://pbs.twimg.com/profile_images/148692830...,shaunlysen,833,1553,81037,53,NaN


In [27]:
export_w_users = export.merge(user_df, on='author_id', how='left')
export_w_users.head()

,text,author_id,in_reply_to_user_id,conversation_id,created_at_x,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,edit_history_tweet_ids,...,verified,description,created_at_y,profile_image_url,username,user.followers_count,user.following_count,user.tweet_count,user.listed_count,withheld.country_codes_y
0,@ywprep @SouthwestAir @IfThenSheCan Am inspire...,1470578806058590209,3.870538e+09,1400856167132368902,2022-09-18T08:01:58.000Z,0,0,0,0,NaN,...,False,|| Climate change Advocate || Nurse ||,2021-12-14T02:18:35.000Z,https://pbs.twimg.com/profile_images/157296401...,tia_tracey,27.0,128.0,471.0,0.0,NaN
1,@FireGregRoman_ @Ravens @SouthwestAir @Patrick...,1332604242675130368,1.330829e+18,1571182195443646464,2022-09-18T07:55:20.000Z,0,0,0,0,NaN,...,False,NaN,2020-11-28T08:36:32.000Z,https://abs.twimg.com/sticky/default_profile_i...,Muggo0,6.0,15.0,1.0,0.0,NaN
2,When I picked up my bag in baggage claim it lo...,47133143,NaN,1571402498178691073,2022-09-18T07:35:25.000Z,0,0,0,0,NaN,...,False,"FanGirl for Potter, VMars, BridgetJones, Ameli...",2009-06-14T17:31:52.000Z,https://pbs.twimg.com/profile_images/117061471...,Nerderific,2165.0,1570.0,88397.0,59.0,NaN
3,"@ILickety @SouthwestAir Yeah, that first one w...",1433359597117530115,1.305605e+18,1569687344067842049,2022-09-18T06:55:22.000Z,0,0,0,0,NaN,...,False,"22, erps as FUTA or female, a rabbit incubus/s...",2021-09-02T09:22:24.000Z,https://pbs.twimg.com/profile_images/149469512...,HarshaLina,66.0,212.0,1350.0,0.0,NaN
4,@JudgeSmailsGolf @tkcaddie @SouthwestAir @lagu...,2991985492,7.361791e+08,1571361678335303680,2022-09-18T06:41:47.000Z,0,0,0,0,NaN,...,False,The only thing necessary for the triumph of ev...,2015-01-22T12:51:38.000Z,https://pbs.twimg.com/profile_images/131126399...,JeffB_jetblast,63.0,1381.0,345.0,0.0,NaN


In [28]:
export_w_users.to_csv('sw-airlines-tweets-w-users.csv', index=True)
upload_to_output('sw-airlines-tweets-w-users.csv', bucket_name, 'data/processed')

In [29]:
export_w_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55287 entries, 0 to 55286
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   text                          55287 non-null  object 
 1   author_id                     55287 non-null  int64  
 2   in_reply_to_user_id           29252 non-null  float64
 3   conversation_id               55287 non-null  int64  
 4   created_at_x                  55287 non-null  object 
 5   public_metrics.retweet_count  55287 non-null  int64  
 6   public_metrics.reply_count    55287 non-null  int64  
 7   public_metrics.like_count     55287 non-null  int64  
 8   public_metrics.quote_count    55287 non-null  int64  
 9   edit_history_tweet_ids        8344 non-null   object 
 10  withheld.copyright            11 non-null     object 
 11  withheld.country_codes_x      11 non-null     object 
 12  tweet_token                   55287 non-null  int64  
 13  n